In [2]:
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt 

Для каждого дня, представленного в таблицах user_actions и courier_actions, рассчитайте следующие показатели:
Число новых пользователей.
Число новых курьеров.
Общее число пользователей на текущий день.
Общее число курьеров на текущий день.

In [11]:
WITH
-- Находим дату первого действия для каждого пользователя
user_start_dates AS ( 
    SELECT
        user_id,
        MIN(time)::date AS start_date
    FROM
        user_actions
    GROUP BY
        user_id
),
-- Считаем количество новых пользователей по дням 
daily_new_users AS (
    SELECT
        start_date,
        COUNT(user_id) AS new_users
    FROM
        user_start_dates
    GROUP BY
        start_date
), 
-- Находим дату первого действия для каждого курьера
courier_start_dates AS (
    SELECT
        courier_id,
        MIN(time)::date AS start_date
    FROM
        courier_actions
    GROUP BY
        courier_id
),
-- Считаем количество новых курьеров по дням 
daily_new_couriers AS (
    SELECT
        start_date,
        COUNT(courier_id) AS new_couriers
    FROM
        courier_start_dates
    GROUP BY
        start_date
)
SELECT
    COALESCE(DNC.start_date, DNU.start_date) AS date,
    COALESCE(DNU.new_users, 0) AS new_users,
    COALESCE(DNC.new_couriers, 0) AS new_couriers,
    SUM(COALESCE(DNU.new_users, 0)) OVER (ORDER BY COALESCE(DNC.start_date, DNU.start_date))::int AS total_users,
    SUM(COALESCE(DNC.new_couriers, 0)) OVER (ORDER BY COALESCE(DNC.start_date, DNU.start_date))::int AS total_couriers
FROM
    daily_new_couriers AS DNC
    FULL JOIN daily_new_users AS DNU
    ON DNC.start_date = DNU.start_date
ORDER BY
    DNC.start_date

SyntaxError: invalid syntax (3080925438.py, line 2)

In [ ]:
file_path1 = ('data/11036746_2025_05_19.csv')
df1 = pd.read_csv(file_path1)
print(df1)

        date  new_users  new_couriers  total_users  total_couriers
0   24/08/22        134            95          134              95
1   25/08/22        829           242          963             337
2   26/08/22       1017           219         1980             556
3   27/08/22       1225           186         3205             742
4   28/08/22       1487           213         4692             955
5   29/08/22       1309           109         6001            1064
6   30/08/22       1204           127         7205            1191
7   31/08/22       1420           195         8625            1386
8   01/09/22       1513           205        10138            1591
9   02/09/22       1887           229        12025            1820
10  03/09/22       1924           161        13949            1981
11  04/09/22       1952           205        15901            2186
12  05/09/22       1397           120        17298            2306
13  06/09/22       1020            98        18318            

Динамика общего числа пользователей. 
![Image](./charts/1.png)

Динамика новых пользователей и курьеров. 
![Image](./charts/2.png)

Количество пользователей, растёт быстрее чем количество курьеров. Показатель числа новых пользователей довольно стабильный, но данные за 6 сентября выбиваются из общей динамики. Показатель числа новых курьеров не дает выбросов и является стабильными на протяжении всех дней. 

Дополните запрос из предыдущего задания и теперь для каждого дня, представленного в таблицах user_actions и courier_actions, дополнительно рассчитайте следующие показатели:
Прирост числа новых пользователей.
Прирост числа новых курьеров.
Прирост общего числа пользователей.
Прирост общего числа курьеров

In [ ]:
WITH
-- Находим дату первого действия для каждого пользователя
user_start_dates AS ( 
    SELECT
        user_id,
        MIN(time)::date AS start_date
    FROM
        user_actions
    GROUP BY
        user_id
),
-- Считаем количество новых пользователей по дням 
daily_new_users AS (
    SELECT
        start_date,
        COUNT(user_id) AS new_users
    FROM
        user_start_dates
    GROUP BY
        start_date
), 
-- Находим дату первого действия для каждого курьера
courier_start_dates AS (
    SELECT
        courier_id,
        MIN(time)::date AS start_date
    FROM
        courier_actions
    GROUP BY
        courier_id
),
-- Считаем количество новых курьеров по дням 
daily_new_couriers AS (
    SELECT
        start_date,
        COUNT(courier_id) AS new_couriers
    FROM
        courier_start_dates
    GROUP BY
        start_date
),
-- Объединяем данные о новых пользователях и курьерах, вычисляем накопительные итоги
user_courier_growth AS (
    SELECT
        COALESCE(DNC.start_date, DNU.start_date) AS date,
        COALESCE(DNU.new_users, 0) AS new_users,
        COALESCE(DNC.new_couriers, 0) AS new_couriers,
        SUM(COALESCE(DNU.new_users, 0)) OVER (ORDER BY COALESCE(DNC.start_date, DNU.start_date))::int AS total_users,
        SUM(COALESCE(DNC.new_couriers, 0)) OVER (ORDER BY COALESCE(DNC.start_date, DNU.start_date))::int AS total_couriers
    FROM
        daily_new_couriers AS DNC
        FULL JOIN daily_new_users AS DNU
        ON DNC.start_date = DNU.start_date
)
SELECT
    date,
    new_users,
    new_couriers,
    total_users,
    total_couriers,
    ROUND(100 * new_users::decimal / LAG(new_users, 1) OVER (ORDER BY date) - 100,2) AS new_users_change,
    ROUND(100 * new_couriers::decimal / LAG(new_couriers, 1) OVER (ORDER BY date) - 100,2) AS new_couriers_change,
    ROUND(100 * total_users::decimal / LAG(total_users, 1) OVER (ORDER BY date) - 100,2) AS total_users_growth,
    ROUND(100 * total_couriers::decimal / LAG(total_couriers, 1) OVER (ORDER BY date) - 100,2) AS total_couriers_growth
FROM
    user_courier_growth
ORDER BY
    date

In [ ]:
file_path2 = ('data/2.1_2025_05_19.csv')
df2 = pd.read_csv(file_path2)
print(df2)

        date  new_users  new_couriers  total_users  total_couriers  \
0   24/08/22        134            95          134              95   
1   25/08/22        829           242          963             337   
2   26/08/22       1017           219         1980             556   
3   27/08/22       1225           186         3205             742   
4   28/08/22       1487           213         4692             955   
5   29/08/22       1309           109         6001            1064   
6   30/08/22       1204           127         7205            1191   
7   31/08/22       1420           195         8625            1386   
8   01/09/22       1513           205        10138            1591   
9   02/09/22       1887           229        12025            1820   
10  03/09/22       1924           161        13949            1981   
11  04/09/22       1952           205        15901            2186   
12  05/09/22       1397           120        17298            2306   
13  06/09/22       1

Динамика прироста числа новых пользователей и курьеров.
![Image](./charts/3.png)

Динамика прироста общего числа пользователей и курьеров.
![Image](./charts/4.png)

Динамика темпа прироста общего числа пользователей и курьеров затухающая. Темп прироста числа новых курьеров опережает темп прироста новых пользователей 31 августа и 7 сентября.

Для каждого дня, представленного в таблицах user_actions и courier_actions, рассчитайте следующие показатели:
Число платящих пользователей.
Число активных курьеров.
Долю платящих пользователей в общем числе пользователей на текущий день.
Долю активных курьеров в общем числе курьеров на текущий день.

In [ ]:
WITH
-- Список отмененных заказов
canceled_orders AS (
    SELECT
        order_id
    FROM 
        user_actions
    WHERE
        action = 'cancel_order'
),
-- Находим дату первого действия для каждого пользователя
user_start_dates AS (
    SELECT
        MIN(time::date) AS start_date,
        user_id
    FROM
        user_actions
    GROUP BY
        user_id
), 
-- Считаем количество новых пользователей по дням 
daily_new_users AS (
    SELECT
        start_date,
        COUNT(user_id) AS new_users
    FROM
        user_start_dates
    GROUP BY
        start_date
),
-- Находим дату первого действия для каждого курьера
courier_start_dates AS (
    SELECT
        MIN(time::date) AS start_date,
        courier_id
    FROM
        courier_actions
    GROUP BY
        courier_id
), 
-- Считаем количество новых курьеров по дням 
daily_new_couriers AS (
    SELECT
        start_date,
        COUNT(courier_id) AS new_couriers
    FROM
        courier_start_dates
    GROUP BY
        start_date
),
-- Объединяем данные о новых пользователях и курьерах, вычисляем накопительные итоги
user_courier_growth AS (
    SELECT
        COALESCE(DNC.start_date, DNU.start_date) AS date, 
        COALESCE(DNU.new_users, 0) AS new_users,
        COALESCE(DNC.new_couriers, 0) AS new_couriers,
        SUM(COALESCE(DNU.new_users, 0)) OVER (ORDER BY COALESCE(DNC.start_date, DNU.start_date))::int AS total_users,
        SUM(COALESCE(DNC.new_couriers, 0)) OVER (ORDER BY COALESCE(DNC.start_date, DNU.start_date))::int AS total_couriers
    FROM
        daily_new_couriers AS DNC
        FULL JOIN daily_new_users AS DNU
        ON DNC.start_date = DNU.start_date
),
-- Считаем активных курьеров по дням (которые доставили заказы)
daily_active_couriers AS (
    SELECT
        time::date AS date,
        COUNT(DISTINCT(courier_id)) AS active_couriers
    FROM
        courier_actions
    WHERE
        order_id NOT IN (SELECT order_id FROM canceled_orders)
    GROUP BY
        date
),
--Считаем платящих пользователей по дням (которые сделали успешные заказы)
daily_paying_users AS (
    SELECT
        time::date AS date,
        COUNT(DISTINCT(user_id)) AS paying_users
    FROM
        user_actions
    WHERE
        order_id NOT IN (SELECT order_id FROM canceled_orders)
    GROUP BY
        date
)
SELECT
    COALESCE(UCG.date,DPU.date) AS date,
    COALESCE(DPU.paying_users, 0) AS paying_users,
    COALESCE(DAC.active_couriers, 0) AS active_couriers,
    ROUND((100 * COALESCE(DPU.paying_users, 0)::decimal / UCG.total_users),2) AS paying_users_share,
    ROUND((100 * COALESCE(DAC.active_couriers, 0)::decimal / UCG.total_couriers),2) AS active_couriers_share
FROM
    user_courier_growth AS UCG
    LEFT JOIN daily_paying_users AS DPU
    ON UCG.date = DPU.date
    LEFT JOIN daily_active_couriers  AS DAC
    ON UCG.date = DAC.date
ORDER BY
    date


In [ ]:
file_path3 = ('data/2025_05_19.csv')
df3 = pd.read_csv(file_path3)
print(df3)

        date  paying_users  active_couriers  paying_users_share  \
0   24/08/22           127               93               94.78   
1   25/08/22           820              333               85.15   
2   26/08/22          1137              526               57.42   
3   27/08/22          1548              721               48.30   
4   28/08/22          2083              927               44.39   
5   29/08/22          2202             1022               36.69   
6   30/08/22          2253             1133               31.27   
7   31/08/22          2430             1286               28.17   
8   01/09/22          2710             1474               26.73   
9   02/09/22          3429             1733               28.52   
10  03/09/22          3794             1906               27.20   
11  04/09/22          4104             2069               25.81   
12  05/09/22          3362             2047               19.44   
13  06/09/22          2701             1883               14.7

Динамика платящих пользователей и активных курьеров.
![Image](./charts/6.png)

Динамика долей платящих пользователей и активных курьеров. 
![Image](./charts/5.png)

Можно сделать вывод, что вместе с общим числом пользователей и курьеров число платящих пользователей и активных курьеров растет. Однако показатели долей активных курьеров очень сильно упала. 

Для каждого дня, представленного в таблице user_actions, рассчитайте следующие показатели:
Долю пользователей, сделавших в этот день всего один заказ, в общем количестве платящих пользователей.
Долю пользователей, сделавших в этот день несколько заказов, в общем количестве платящих пользователей.

In [ ]:
WITH
-- Список отмененных заказов
canceled_orders AS (
    SELECT
        order_id
    FROM
        user_actions
    WHERE
        action = 'cancel_order'
),
-- Все действия пользователей с неотмененными заказами
valid_user_actions AS (
    SELECT
        time::date AS date,
        user_id,
        order_id
    FROM
        user_actions
    WHERE
        order_id NOT IN (SELECT order_id FROM canceled_orders)
),
-- Платящие пользователи по дням
paying_users_daily AS (
    SELECT
        date, 
        COUNT(DISTINCT(user_id)) AS paying_users
    FROM
        valid_user_actions
    GROUP BY
        date
),
-- Количество заказов для каждого пользователя в каждый день
user_orders_count AS (
    SELECT
        date,
        user_id,
        COUNT(DISTINCT(order_id)) AS orders_count
    FROM
        valid_user_actions
    GROUP BY
        date,
        user_id
),
-- Пользователи с ровно одним заказов по дням
single_order_users_daily AS (
    SELECT
        date, 
        user_id,
        COUNT(user_id) AS single_order_users
    FROM
        user_orders_count
    WHERE
        orders_count = 1
    GROUP BY
        date,
        user_id
)
SELECT
    COALESS(PUD.date, SOUD.date) AS date,
    ROUND((100 * SOUD.single_order_users::decimal / PUD.paying_users),2) AS single_order_users_share,
    100 - ROUND((100 * SOUD.single_order_users::decimal / PUD.paying_users),2) AS several_orders_users_share
FROM
    paying_users_daily AS PUD
    LEFT JOIN single_order_users_daily AS SOUD 
    ON PUD.date = SOUD.date
ORDER BY    
    PUD.date

In [ ]:
file_path4 = ('data/66.csv')
df4 = pd.read_csv(file_path4)
print(df4)

        date  singe_order_users_share  several_orders_users_share
0   24/08/22                    92.91                        7.09
1   25/08/22                    76.46                       23.54
2   26/08/22                    78.45                       21.55
3   27/08/22                    70.93                       29.07
4   28/08/22                    67.83                       32.17
5   29/08/22                    65.58                       34.42
6   30/08/22                    64.67                       35.33
7   31/08/22                    70.41                       29.59
8   01/09/22                    71.70                       28.30
9   02/09/22                    67.02                       32.98
10  03/09/22                    63.94                       36.06
11  04/09/22                    65.94                       34.06
12  05/09/22                    69.96                       30.04
13  06/09/22                    77.67                       22.33
14  07/09/

Доля пользователей с одним и несколькими заказами. 
![Image](./charts/7.png)

Аномально низким значение показателя в перый день назвать нельзя. 

Для каждого дня, представленного в таблице user_actions, рассчитайте следующие показатели:
Общее число заказов.
Число первых заказов (заказов, сделанных пользователями впервые).
Число заказов новых пользователей (заказов, сделанных пользователями в тот же день, когда они впервые воспользовались сервисом).
Долю первых заказов в общем числе заказов (долю п.2 в п.1).
Долю заказов новых пользователей в общем числе заказов (долю п.3 в п.1).

In [ ]:
WITH
-- Список отмененных заказов
canceled_orders AS (
    SELECT
        order_id
    FROM
        user_actions
    WHERE
        action = 'cancel_order'
),
-- Список не отмененных заказов
valid_user_orders AS (
    SELECT
        time::date AS date,
        user_id,
        order_id
    FROM
        user_actions
    WHERE
        order_id NOT IN (SELECT order_id FROM canceled_orders)
),
-- Количество заказов по дням (исключая отменённые)
daily_orders AS (
    SELECT
        COUNT(DISTINCT(order_id))::int AS orders,
        time::date AS date
    FROM
        user_actions
    WHERE
        action = 'create_order' 
        AND order_id NOT IN (SELECT order_id FROM canceled_orders)    
    GROUP BY
        date
),
-- Даты первых заказов для каждого пользователя (неотмененные)
user_first_orders AS (
    SELECT
        MIN(time)::date AS first_order_date,
        user_id
    FROM
        user_actions
    WHERE 
        action = 'create_order' 
        AND order_id NOT IN (SELECT order_id FROM canceled_orders) 
    GROUP BY
        user_id
),
-- Количество первых заказов по дням
daily_first_orders AS (
    SELECT
        COUNT(user_id)::int AS first_orders,
        first_order_date
    FROM
        user_first_orders
    GROUP BY
        first_order_date
),
-- Заказы сделанные пользователями в первый день использования сервиса
user_first_actions AS (
    SELECT
        user_id,
        MIN(time)::date AS first_action_date
    FROM
        user_actions
    GROUP BY
        user_id
),
-- Заказы, сделанные пользователями в первый день использования сервиса
new_users_daily_orders AS (
    SELECT
        COALESCE(UFA.first_action_date,YUO.date) AS date,
        COUNT(VUO.order_id)::int AS new_users_orders
    FROM
        user_first_actions AS UFA
        LEFT JOIN valid_user_orders AS VUO
        ON UFA.first_action_date = VUO.date AND UFA.user_id = VUO.user_id
    GROUP BY
        UFA.first_action_date
)
SELECT
    COALESCE(NUDO.date, DFO.date) AS date,
    COALESCE(DO1.orders, 0) AS orders,
    COALESCE(DFO.first_orders, 0) AS first_orders,
    COALESCE(NUDO.new_users_orders, 0) AS new_users_orders,
    ROUND((100 * COALESCE(DFO.first_orders,0)::decimal / DO1.orders),2) AS first_orders_share,
    ROUND((100 * COALESCE(NUDO.new_users_orders,0)::decimal / DO1.orders),2) AS new_users_orders_share
FROM
    daily_first_orders AS DFO
    LEFT JOIN new_users_daily_orders AS NUDO
    ON NUDO.date = DFO.first_order_date
    LEFT JOIN daily_orders AS DO1
    ON DO1.date = DFO.first_order_date
ORDER BY
    date

In [12]:
file_path5 = ('data/111.csv')
df5 = pd.read_csv(file_path5)
print(df5)

        date  orders  first_orders  new_users_orders  first_orders_share  \
0   24/08/22     138           127               138               92.03   
1   25/08/22    1059           802              1032               75.73   
2   26/08/22    1447           984              1250               68.00   
3   27/08/22    2141          1192              1624               55.67   
4   28/08/22    2998          1460              2102               48.70   
5   29/08/22    3267          1281              1832               39.21   
6   30/08/22    3371          1180              1714               35.00   
7   31/08/22    3410          1380              1908               40.47   
8   01/09/22    3688          1492              1988               40.46   
9   02/09/22    5001          1864              2655               37.27   
10  03/09/22    5709          1907              2830               33.40   
11  04/09/22    6010          1943              2763               32.33   
12  05/09/22

Динамика общего числа заказов, числа первых заказов и числа заказов новых пользователей. 
![Image](./charts/8.png)

Динамика доли первых заказов и доли заказов новых пользователей в общем числе заказов. 
![Image](./charts/9.png)

С ростом количества всех заказов растут показатели числа первых заказов и числа заказов новых пользователей. 

На основе данных в таблицах user_actions, courier_actions и orders для каждого дня рассчитайте следующие показатели:
Число платящих пользователей на одного активного курьера.
Число заказов на одного активного курьера.

In [ ]:
WITH
-- Список отмененных заказов
canceled_orders AS (
    SELECT
        order_id
    FROM
        user_actions
    WHERE
        action = 'cancel_order'
),
-- Находим дату первого действия для каждого пользователя
user_start_dates AS (
    SELECT
        user_id,
        MIN(time)::date AS start_date
    FROM
        user_actions
    GROUP BY
        user_id
),
-- Считаем количество новых пользователей по дням 
daily_new_users AS (
    SELECT
        start_date,
        COUNT(user_id) AS new_users
    FROM
        user_start_dates
    GROUP BY
        start_date
),
-- Количесвто платящих пользователей по дням
daily_paying_users AS (
    SELECT
        time::date AS date,
        COUNT(DISTINCT(user_id)) AS paying_users
    FROM
        user_actions
    WHERE 
        order_id NOT IN (SELECT order_id FROM canceled_orders)
    GROUP BY
        date
), 
-- Количество активных курьеров по дням
daily_active_couriers AS (
    SELECT
        time::date AS date,
        COUNT(DISTINCT(courier_id)) AS active_couriers
    FROM
        courier_actions
    WHERE
        order_id NOT IN (SELECT order_id FROM canceled_orders)
    GROUP BY
        date
),
-- Ежедневное количество заказов
daily_orders AS (
    SELECT
        creation_time::date AS date,
        COUNT(DISTINCT(order_id)) AS orders_count
    FROM
        orders
    WHERE
        order_id NOT IN (SELECT order_id FROM canceled_orders)
    GROUP BY
        date
)
SELECT
    COALESCE(DPU.date, DAC.date) AS date,
    ROUND(DPU.paying_users::decimal / DAC.active_couriers, 2) AS users_per_courier,
    ROUND(DO1.orders_count::decimal / DAC.active_couriers, 2) AS orders_per_courier
FROM
    daily_paying_users AS DPU
    LEFT JOIN daily_active_couriers AS DAC
    ON DPU.date = DAC.date
    LEFT JOIN daily_orders AS DO1
    ON DPU.date = DO1.date
ORDER BY
    date

In [3]:
file_path6 = ('data/666.csv')
df6 = pd.read_csv(file_path6)
print(df6)

        date  users_per_courier  orders_per_courier
0   24/08/22               1.37                1.48
1   25/08/22               2.46                3.18
2   26/08/22               2.16                2.75
3   27/08/22               2.15                2.97
4   28/08/22               2.25                3.23
5   29/08/22               2.15                3.20
6   30/08/22               1.99                2.98
7   31/08/22               1.89                2.65
8   01/09/22               1.84                2.50
9   02/09/22               1.98                2.89
10  03/09/22               1.99                3.00
11  04/09/22               1.98                2.90
12  05/09/22               1.64                2.28
13  06/09/22               1.43                1.83
14  07/09/22               1.57                2.13
15  08/09/22               1.57                2.20


Динамика числа пользователей и заказов на одного курьера. 
![Image](./charts/10.png)

Динамика числа пользователей и заказов на одного курьера совпадает. Так как эти метрики обычно коррелируют между собой - чем больше пользователей, тем больше заказов они создают. Сначала был рост показателей, но с течением времени значение показателей стало снижаться. 

На основе данных в таблице courier_actions для каждого дня рассчитайте, за сколько минут в среднем курьеры доставляли свои заказы.

In [ ]:
WITH
canceled_order AS (
    SELECT
        order_id
    FROM
        user_actions
    WHERE
        action = 'cancel_order'
),
order_dates AS (
    SELECT
        order_id,
        time::date AS date
    FROM 
        courier_actions
    WHERE
        order_id NOT IN (SELECT order_id FROM canceled_order)
),
order_time_range AS (
    SELECT
        order_id,
        MIN(time) AS start_time,
        MAX(time) AS end_time
    FROM
        courier_actions
    WHERE
        order_id NOT IN (SELECT order_id FROM canceled_order)
    GROUP BY
        order_id
),
delivery_times AS (
    SELECT
        OD1.date,
        EXTRACT(epoch FROM (OTR.end_time - OTR.start_time)) / 60 AS delivery_minutes
    FROM
        order_dates AS OD1
        JOIN order_time_range OTR 
        ON OD1.order_id = OTR.order_id
)
SELECT
    date,
    ROUND(AVG(delivery_minutes))::int AS minutes_to_deliver
FROM
    delivery_times
GROUP BY
    date
ORDER BY
    date

In [5]:
file_path7 = ('data/333.csv')
df7 = pd.read_csv(file_path7)
print(df7)

        date  minutes_to_deliver
0   24/08/22                  20
1   25/08/22                  20
2   26/08/22                  20
3   27/08/22                  20
4   28/08/22                  20
5   29/08/22                  20
6   30/08/22                  20
7   31/08/22                  20
8   01/09/22                  20
9   02/09/22                  20
10  03/09/22                  20
11  04/09/22                  20
12  05/09/22                  20
13  06/09/22                  20
14  07/09/22                  20
15  08/09/22                  20


Динамика среднего времени доставки заказов.
![Image](./charts/12.png)

У курьеров получается придерживаться в среднем одинакового времени доставки заказов. 

На основе данных в таблице orders для каждого часа в сутках рассчитайте следующие показатели:
Число успешных (доставленных) заказов.
Число отменённых заказов.
Долю отменённых заказов в общем числе заказов (cancel rate).

In [ ]:
WITH
canceled_orders AS (
    SELECT
        order_id
    FROM
        user_actions
    WHERE
        action = 'cancel_order'
),
delivered_orders AS (
    SELECT
        order_id
    FROM
        courier_actions
    WHERE
        action = 'deliver_order'
), 
t1 AS (
    SELECT
        DATE_PART('hour', creation_time)::int AS hour,
        COUNT(order_id) FILTER (WHERE order_id IN (SELECT order_id FROM delivered_orders)) AS successful_orders,
        COUNT(order_id) FILTER (WHERE order_id IN (SELECT order_id FROM canceled_orders)) AS canceled_orders,
        COUNT(order_id) AS order_count
    FROM
        orders
    GROUP BY
        hour
)
SELECT
    hour, 
    successful_orders,
    canceled_orders,
    ROUND(canceled_orders::decimal / order_count, 3) AS cancel_rate
FROM 
    t1
ORDER BY
    hour    

In [6]:
file_path8 = ('data/665.csv')
df8 = pd.read_csv(file_path8)
print(df8)

    hour  successful_orders  canceled_orders  cancel_rate
0      0               2297              139        0.057
1      1               1867               98        0.050
2      2               1422               62        0.042
3      3                958               56        0.055
4      4                926               48        0.049
5      5               1363               77        0.053
6      6               1583              101        0.060
7      7               2101              100        0.045
8      8               2195              117        0.051
9      9               2606              151        0.055
10    10               2641              135        0.049
11    11               2584              121        0.045
12    12               2440              124        0.048
13    13               2424              122        0.048
14    14               2351              120        0.049
15    15               2513              128        0.048
16    16      

Динамика показателя cancel rate и числа успешных/отменённых заказов.
![Image](./charts/13.png)

В промежуток времени между 19:00 - 20:00 пользователи осуществялют максимальное количество заказов. Меньше всего заказов совершают в промежутке с 3:00 - 5:00 утра. Показатели cancel rate растет с увеличением числа заказов. 